In [1]:
## make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))

%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np
from dataclasses import dataclass


from pa_lib.file import (
    project_dir,
    load_bin,
    load_pickle,
    load_xlsx,
    store_bin,
)
from pa_lib.data import (
    as_dtype,
    dtFactor,
    lookup,
    desc_col,
    chi2_expected,
    clean_up_categoricals,
)
from pa_lib.util import cap_words, collect, value, normalize_rows, as_percent, flatten

# display long columns completely, show more rows
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

# Load data sets

In [2]:
with project_dir("axinova"):
    ax_data = load_bin("ax_data.feather")
    spr_data = load_pickle("spr_data.pkl")
    time_codes = load_pickle("time_code_ratios.pkl")
    station_codes = load_pickle("station_code_ratios.pkl")
    global_codes = load_pickle("global_code_ratios.pkl")
    population_codes = load_pickle("population_ratios.pkl")

11:36:11 [INFO] Started loading binary file ...
11:36:11 [INFO] Reading from file C:\Users\kpf\data\axinova\ax_data.feather
11:36:12 [INFO] ... finished loading binary file in 0.31s (0.95s CPU)
11:36:12 [INFO] Started loading pickle file ...
11:36:12 [INFO] Reading from file C:\Users\kpf\data\axinova\spr_data.pkl
11:36:12 [INFO] ... finished loading pickle file in 0.01s (0.02s CPU)
11:36:12 [INFO] Started loading pickle file ...
11:36:12 [INFO] Reading from file C:\Users\kpf\data\axinova\time_code_ratios.pkl
11:36:12 [INFO] ... finished loading pickle file in 0.03s (0.0s CPU)
11:36:12 [INFO] Started loading pickle file ...
11:36:12 [INFO] Reading from file C:\Users\kpf\data\axinova\station_code_ratios.pkl
11:36:12 [INFO] ... finished loading pickle file in 0.0s (0.0s CPU)
11:36:12 [INFO] Started loading pickle file ...
11:36:12 [INFO] Reading from file C:\Users\kpf\data\axinova\global_code_ratios.pkl
11:36:12 [INFO] ... finished loading pickle file in 0.0s (0.0s CPU)
11:36:12 [INFO] St

## Select Axinova data by any column(s)

In [4]:
def _check_selection(data, selection, allowed_columns):
    allowed_values = {}
    for column in allowed_columns:
        allowed_values[column] = data[column].cat.categories
    if set(selection.keys()) - set(allowed_columns) != set():
        raise NameError(f"Unknown column name in selection: {selection.keys()}")
    clean_selection = {}
    for column in allowed_columns:
        if column in selection:
            col_values = list(flatten(selection[column]))
            if set(col_values) - set(allowed_values[column]) != set():
                raise ValueError(
                    f"Illegal value(s) in parameter {column}: {col_values}"
                )
            clean_selection[column] = col_values
        else:
            clean_selection[column] = None
    return clean_selection


def select_data(all_data, **selection):
    select_columns = (
        "DayOfWeek Station Variable Month TimeSlot Hour "
        + "Time TimeSlot_cat StationSprache".split()
    )
    selection = _check_selection(all_data, selection, allowed_columns=select_columns)
    row_mask = pd.Series([True] * all_data.shape[0])
    for col in select_columns:
        if selection[col] is not None:
            row_mask &= all_data[col].isin(selection[col])
    return all_data.loc[row_mask].pipe(clean_up_categoricals).reset_index(drop=True)

# Look up code ratios from Axinova data

In [5]:
RatioTable = pd.DataFrame


@dataclass
class Ratios:
    actual: RatioTable
    expected: RatioTable
    sd: RatioTable = None

In [ ]:
def ax_population_ratios(variable: str, percent: bool = False) -> RatioTable:
    ratios = population_codes.query("Variable == @variable").pivot_table(
        values="Pop_Ratio", index="Variable", columns="Code"
    )
    return as_percent(ratios) if percent else ratios


def ax_global_ratios(variable: str, percent: bool = False) -> RatioTable:
    ratios = global_codes.query("Variable == @variable").pivot_table(
        values="Ratio", index="Variable", columns="Code"
    )
    return as_percent(ratios) if percent else ratios


def ax_station_ratios(variable: str, percent: bool = False) -> Ratios:
    actual_ratios = station_codes.query("Variable == @variable").pivot_table(
        values="Ratio", index="Station", columns="Code", fill_value=0
    )
    expected_ratios = ax_global_ratios(variable)
    if percent:
        result = Ratios(
            actual=as_percent(actual_ratios), expected=as_percent(expected_ratios)
        )
    else:
        result = Ratios(actual=actual_ratios, expected=expected_ratios)
    return result

In [6]:
def ax_ratios(
    variable: str,
    stations: str,
    reference: str = "all_stations",
    time_scale: str = "Hour",
    percent: bool = False,
) -> Ratios:
    subset = ax_data.loc[
        ax_data.Station.isin(flatten(stations)) & (ax_data.Variable == variable)
    ]
    full_index = [
        (weekday, time)
        for weekday in ax_data["DayOfWeek"].cat.categories
        for time in ax_data[time_scale].cat.categories
    ]
    actual_counts = subset.pivot_table(
        values="Value",
        index=["DayOfWeek", time_scale],
        columns="Code",
        fill_value=0,
        aggfunc="sum",
    )
    actual_counts_sd_ratios = (
        (np.sqrt(actual_counts) / actual_counts)
        .fillna(0)
        .reindex(full_index, fill_value=0)
    )
    actual_ratios = normalize_rows(actual_counts).reindex(full_index, fill_value=0)

    if reference == "all_stations":
        expected_ratios = (
            time_codes[time_scale]
            .query("Variable == @variable")
            .pivot_table(
                values="Ratio",
                index=["DayOfWeek", time_scale],
                columns="Code",
                fill_value=0,
            )
        )
    elif reference == "station":
        expected_counts = chi2_expected(actual_counts)
        expected_ratios = normalize_rows(expected_counts).reindex(
            full_index, fill_value=0
        )
    else:
        raise ValueError(
            "Parameter 'reference' must be one of "
            + f"('station', 'all_stations'), was '{reference}'"
        )

    if percent:
        result = Ratios(
            actual=as_percent(actual_ratios),
            expected=as_percent(expected_ratios),
            sd=as_percent(actual_counts_sd_ratios),
        )
    else:
        result = Ratios(
            actual=actual_ratios, expected=expected_ratios, sd=actual_counts_sd_ratios
        )
    return result

# Look up SPR+ data split by variable

In [7]:
def spr_split(
    stations,
    variable,
    reference="station",
    time_scale="Hour",
    type="abs",
    incl_totals=True,
    decimals=1,
):
    if type not in ["abs", "rel"]:
        raise ValueError(
            f"Parameter 'type' must be one of ('abs', 'rel'), was '{type}'"
        )
    if reference not in ["station", "all_stations"]:
        raise ValueError(
            "Parameter 'reference' must be one of "
            + f"('station', 'all_stations'), was '{reference}'"
        )

    spr_counts = (
        spr_data.loc[spr_data.Station.isin(flatten(stations))]
        .groupby(["DayOfWeek", time_scale])[["Total"]]
        .agg("sum")
    )
    spr_count_sd_ratios = (np.sqrt(spr_counts) / spr_counts).fillna(0)

    ratios = ax_ratios(
        stations=stations,
        variable=variable,
        reference=reference,
        time_scale=time_scale,
        percent=False,
    )
    if type == "abs":
        code_ratios = ratios.actual
    elif type == "rel":
        code_ratios = ratios.actual - ratios.expected
    split_counts = code_ratios.mul(spr_counts.Total, axis="index").round(decimals)

    sd_ratios = np.sqrt(
        (ratios.sd ** 2).add((spr_count_sd_ratios ** 2).values, axis="index")
    )

    if incl_totals:
        split_counts.set_axis(split_counts.columns.to_list(), axis="columns", inplace=True)
        split_counts["Total"] = spr_counts.round(decimals)
        sd_ratios.set_axis(sd_ratios.columns.to_list(), axis="columns", inplace=True)
        sd_ratios["Total"] = spr_count_sd_ratios.values
    if decimals == 0:
        split_counts = split_counts.astype("int")

    return split_counts, sd_ratios

# Tests

## Station ratios

In [ ]:
with value(ax_station_ratios(variable="g_220", percent=True)) as autobesitz:
    display(autobesitz.actual)
    display(autobesitz.expected)
    display(autobesitz.actual.sub(autobesitz.expected.values, axis="columns"))

## Time ratios for one station

In [ ]:
with value(ax_ratios(stations="Lausanne", variable="g_220")) as lausanne_auto:
    display(lausanne_auto.actual)
    display(lausanne_auto.expected)
    display(lausanne_auto.actual - lausanne_auto.expected)

## SPR+ data for one station split by ratios

In [10]:
with value(("Bern", "g_220", "TimeSlot")) as (stat, var, scale):
    display(spr_split(stations=stat, variable=var, type="abs", time_scale=scale)[0])
    display(spr_split(stations=stat, variable=var, type="rel", time_scale=scale)[0])

1 Auto  2+ Autos   Keines     Total
DayOfWeek TimeSlot                                         
Monday    Nacht         8193.2    1815.5   3059.3   13068.0
          Morgen-Rush  43950.4   33045.6  30790.9  107786.9
          Morgen       22057.4   14845.0  16399.0   53301.5
          Mittag       25993.5   15503.8  24558.9   66056.2
          Nachmittag   40167.5   24950.9  31855.5   96973.8
          Abend-Rush   73300.6   49895.3  44452.1  167647.9
          Abend        34653.0   17017.2  35876.0   87546.3
Tuesday   Nacht         7469.0    3562.4   1912.2   12943.6
          Morgen-Rush  47872.6   30540.9  32672.6  111086.1
          Morgen       18395.7   14999.0  17783.9   51178.6
          Mittag       28423.3   20512.7  14572.2   63508.2
          Nachmittag   38133.9   31056.2  24804.0   93994.1
          Abend-Rush   74238.4   57801.1  39045.7  171085.2
          Abend        40848.7   19744.0  30027.1   90619.8
Wednesday Nacht         8805.2    2725.7   2292.1   13823.0
          Morgen-Rush  43944.5   28991.6  33123.3  106059.4
          Morgen       15147.3   20366.9  14765.9   50280.1
          Mittag       23601.6   20558.8  20611.4   64771.7
          Nachmittag   37711.4   32812.8  26289.9   96814.2
          Abend-Rush   74253.1   51599.3  42241.1  168093.5
          Abend        47254.6   16944.1  28662.7   92861.5
Thursday  Nacht         9584.4    2807.9   3022.3   15414.6
          Morgen-Rush  44832.5   24345.4  35531.5  104709.5
          Morgen       17764.0   15658.9  17256.9   50679.8
          Mittag       22601.6   19865.9  21105.3   63572.8
          Nachmittag   40325.8   29773.0  25291.3   95390.1
          Abend-Rush   74709.3   45841.6  47174.3  167725.2
          Abend        43226.9   22239.2  31240.0   96706.1
Friday    Nacht        10162.7    6375.9   5225.0   21763.6
          Morgen-Rush  41683.2   24464.8  23567.8   89715.8
          Morgen       22732.7   12544.0  13096.6   48373.3
          Mittag       29840.3   16880.9  15761.1   62482.3
          Nachmittag   43825.9   26574.6  30133.8  100534.3
          Abend-Rush   72029.4   53202.3  37582.1  162813.8
          Abend        42615.3   25419.7  38015.5  106050.5
Saturday  Nacht        12827.2   12887.0   6477.1   32191.4
          Morgen-Rush  15907.1   14936.4  14114.2   44957.7
          Morgen       22664.4   17343.8  15576.8   55585.0
          Mittag       32654.9   20433.2  18594.2   71682.3
          Nachmittag   65678.1   36035.9  24372.1  126086.2
          Abend-Rush   62375.4   39596.3  38417.3  140389.0
          Abend        55034.2   29498.3  27955.1  112487.6
Sunday    Nacht         9676.6   12292.7   4714.6   26683.9
          Morgen-Rush  18886.4    4787.8   6980.2   30654.4
          Morgen       21431.2   11665.8  17256.8   50353.8
          Mittag       31904.0   15750.7  24346.8   72001.6
          Nachmittag   68945.0   24318.4  41891.8  135155.2
          Abend-Rush   70277.7   38936.6  44640.4  153854.8
          Abend        52715.3   25774.3  28481.7  106971.3

1 Auto  2+ Autos   Keines     Total
DayOfWeek TimeSlot                                         
Monday    Nacht         2529.3   -1877.6   -651.7   13068.0
          Morgen-Rush  -2766.1    2584.6    181.5  107786.9
          Morgen       -1044.2    -218.2   1262.4   53301.5
          Mittag       -2636.3   -3163.9   5800.1   66056.2
          Nachmittag   -1862.4   -2454.3   4316.7   96973.8
          Abend-Rush     639.5    2517.3  -3156.8  167647.9
          Abend        -3290.8   -7723.7  11014.5   87546.3
Tuesday   Nacht         1859.1     -95.5  -1763.5   12943.6
          Morgen-Rush   -273.8    -852.4   1126.2  111086.1
          Morgen       -3785.9     535.7   3250.1   51178.6
          Mittag         897.9    2565.0  -3463.0   63508.2
          Nachmittag   -2604.6    4493.2  -1888.6   93994.1
          Abend-Rush      87.5    9451.8  -9539.3  171085.2
          Abend         1572.7   -5865.5   4292.8   90619.8
Wednesday Nacht         2814.1   -1180.7  -1633.4   13823.0
          Morgen-Rush  -2023.2    -981.2   3004.4  106059.4
          Morgen       -6644.9    6157.6    487.3   50280.1
          Mittag       -4471.5    2254.0   2217.5   64771.7
          Nachmittag   -4249.3    5452.8  -1203.5   96814.2
          Abend-Rush    1398.9    4095.5  -5494.3  168093.5
          Abend         7007.0   -9298.9   2291.8   92861.5
Thursday  Nacht         2903.5   -1548.3  -1355.2   15414.6
          Morgen-Rush   -550.1   -5245.9   5796.0  104709.5
          Morgen       -4201.4    1336.6   2864.8   50679.8
          Mittag       -4951.8    1900.0   3051.8   63572.8
          Nachmittag   -1017.7    2815.4  -1797.7   95390.1
          Abend-Rush    2014.7   -1558.2   -456.5  167725.2
          Abend         1313.1   -5090.3   3777.2   96706.1
Friday    Nacht          730.1     225.4   -955.4   21763.6
          Morgen-Rush   2799.0    -889.2  -1909.8   89715.8
          Morgen        1767.0   -1126.5   -640.5   48373.3
          Mittag        2759.5    -776.8  -1982.8   62482.3
          Nachmittag     252.8   -1836.8   1584.0  100534.3
          Abend-Rush    1463.4    7190.6  -8654.0  162813.8
          Abend        -3348.6   -4550.6   7899.1  106050.5
Saturday  Nacht        -1125.0    3789.6  -2664.6   32191.4
          Morgen-Rush  -3578.3    2231.2   1347.0   44957.7
          Morgen       -1427.0    1635.3   -208.3   55585.0
          Mittag        1586.7     175.6  -1762.3   71682.3
          Nachmittag   11030.5     403.5 -11434.0  126086.2
          Abend-Rush    1528.7     -78.2  -1450.6  140389.0
          Abend         6280.4   -2291.1  -3989.3  112487.6
Sunday    Nacht        -1888.6    4751.7  -2863.1   26683.9
          Morgen-Rush   5600.3   -3875.3  -1725.1   30654.4
          Morgen        -392.9   -2564.3   2957.3   50353.8
          Mittag         697.5   -4597.2   3899.7   72001.6
          Nachmittag   10366.7  -13876.9   3510.2  135155.2
          Abend-Rush    3594.8   -4543.3    948.5  153854.8
          Abend         6352.4   -4456.2  -1896.1  106971.3

# Case study: Find best stations / time slots for owner of multiple cars

## Population ratios

In [ ]:
ax_population_ratios(variable="g_220", percent=True)

## Ratios over all stations

In [ ]:
ax_global_ratios(variable="g_220", percent=True)

Multi-car owners are **underrepresented** at digital station panels (29% vs. 40%)

## Ratios per station

In [ ]:
ax_station_ratios(variable="g_220", percent=True).actual

Best stations are **Fribourg** (56%), **Zürich Flughafen** (50%), and **Genève Aéroport** (49%).

This compares to 29% over all stations, and 40% in the base population.

## Time slots at one station: Zürich HB

In [ ]:
prozent = ax_ratios(
    variable="g_220", stations="Zürich Flughafen", time_scale="Hour", percent=True
).actual
personen = spr_split(
    stations="Zürich HB", variable="g_220", time_scale="Hour", type="abs", decimals=0
)[0]
time_analysis = prozent.merge(
    personen, left_index=True, right_index=True, suffixes=(" [%]", " [Pers.]")
)

time_analysis.loc[prozent["2+ Autos"] > 40]

In [ ]:
time_analysis.sort_values("2+ Autos [Pers.]", ascending=False).head(20)

## Time slots with CI at one station: Zürich Flughafen

In [13]:
(counts, sd_ratios) = spr_split(
    stations="Zürich Flughafen", variable="g_220", time_scale="TimeSlot", type="abs", decimals=1
)

counts.merge(
    as_percent(sd_ratios), left_index=True, right_index=True, suffixes=(" [Pers]", " [sd%]")
)

1 Auto [Pers]  2+ Autos [Pers]  Keines [Pers]  \
DayOfWeek TimeSlot                                                     
Monday    Nacht                 38.0             99.7           13.9   
          Morgen-Rush          400.6            630.7          222.6   
          Morgen               239.8            373.4            7.8   
          Mittag               273.8            444.7           51.8   
          Nachmittag           301.4            771.8           58.6   
          Abend-Rush           818.6            785.4          353.1   
          Abend                310.7            328.1          382.5   
Tuesday   Nacht                 23.8            106.2           20.7   
          Morgen-Rush          313.4            901.9           81.5   
          Morgen               194.4            320.5           82.0   
          Mittag               294.5            316.7          129.5   
          Nachmittag           296.3            705.7           94.2   
          Abend-Rush           765.3            905.0          324.0   
          Abend                450.2            382.6          222.7   
Wednesday Nacht                 70.2             60.2           28.7   
          Morgen-Rush          258.4            701.1          275.2   
          Morgen               209.4            339.6           36.7   
          Mittag               313.4            365.4           76.6   
          Nachmittag           437.0            624.7           67.0   
          Abend-Rush           585.0           1034.9          343.3   
          Abend                342.9            560.7          178.3   
Thursday  Nacht                 44.8             88.0           46.3   
          Morgen-Rush          668.1            340.6          211.5   
          Morgen               269.4            253.7           67.6   
          Mittag               346.2            287.1          108.9   
          Nachmittag           329.4            665.8          117.9   
          Abend-Rush           727.9           1081.6          148.5   
          Abend                389.6            361.0          378.8   
Friday    Nacht                 30.1            202.0           21.7   
          Morgen-Rush          399.9            533.1          113.7   
          Morgen               259.4            257.9           46.7   
          Mittag               236.2            387.6          104.8   
          Nachmittag           438.2            563.6          171.5   
          Abend-Rush           790.5            834.5          275.8   
          Abend                485.4            619.3          130.5   
Saturday  Nacht                196.5            174.4            5.9   
          Morgen-Rush          211.4            275.9           36.5   
          Morgen               274.1            230.3          143.8   
          Mittag               330.9            264.4          239.4   
          Nachmittag           524.4            754.9          192.9   
          Abend-Rush           530.9            909.5          197.5   
          Abend                550.6            532.8          229.1   
Sunday    Nacht                 88.7            198.0           22.3   
          Morgen-Rush          172.9            150.1           33.1   
          Morgen               128.9            381.7           77.1   
          Mittag               187.8            494.9          158.7   
          Nachmittag           307.0            814.9          454.7   
          Abend-Rush           703.2            658.7          433.6   
          Abend                566.9            454.1          225.6   

                       Total [Pers]  1 Auto [sd%]  2+ Autos [sd%]  \
DayOfWeek TimeSlot                                                  
Monday    Nacht               151.6            26              18   
          Morgen-Rush        1253.9            20              16   
          Morgen              621.0            18              15   
     

## Analysis for ZRH Airport (ax_data only)

In [ ]:
as_percent(ax_ratios(
    variable="g_220",
    stations="Zürich Flughafen - Airside",
    reference="station",
    time_scale="Hour",
).actual)